In [ ]:
import sys
sys.path.append('../')

import random
import json
import os
from tqdm import tqdm
import time
from utils import create_driver
import pandas as pd

from data_sources.google_scholar import scrape_publication_details, save_publications_details

In [ ]:
raw_dr_ntu_dir='./raw_dr_ntu'
raw_faculty_db = 'scse_profile'

process_faculty_db_dir ='./processed'
process_faculty_db = 'scse_profile'

raw_google_scholar_dir = './raw_google_scholar'
raw_google_search_dir = './google_search'

In [ ]:
faculties = pd.read_csv(os.path.join(process_faculty_db_dir, process_faculty_db+'.csv'))
google_scholar_faculties = faculties[faculties['google_scholar'].notna()]

# Scrape google scholar publications

In [ ]:
driver = create_driver(debug=True)

for i, row in tqdm(google_scholar_faculties.iterrows()):
    name, id = row['full_name'], row['google_scholar_id']

    with open(f'{raw_google_scholar_dir}/{id}.json', 'r') as f:
        profile = json.load(f)

    publications = profile['publications']

    details = {}
    for i, pub in tqdm(enumerate(publications)):
        pub_link = pub['title_link']
        driver.get(pub_link)
        time.sleep(random.uniform(1,2))
        details[pub_link] = scrape_publication_details(pub_link, driver)

        if i%10==0:
            save_publications_details(details, f'{id}_publications.json', output_dir=raw_google_scholar_dir)

        if random.uniform(0,1)>0.8:
            time.sleep(random.uniform(5,7))

        if random.uniform(0,1)>0.9:
            driver.quit()
            driver = create_driver(debug=False)

    save_publications_details(details, f'{id}_publications.json', output_dir=raw_google_scholar_dir)
    driver.quit()
    driver = create_driver(debug=False)

driver.quit()
